# 项目6-异常检测

## 友情提示

同学们可以前往课程作业区先行动手尝试！！！

## 项目描述

这份作业要做的是半监督异常检测，也就是说训练集是干净的，测试的时候才会混进超限数据（异常）。 我们以某个简单的图像数据集（图像加上他们的分类）作为示范，训练集为原先训练中的某几类，而测试集则是原先测试集的所有数据，要侦测的异常为训练集中未出现的类别。 标签的部分，1 为超限数据（异常），而 0 为正常数据。   
最终要实现：在只给定干净的（无异常）训练集的情况下，分辨测试集中哪些数据是来自训练集或是从未见过的类别。

## 数据集介绍

* Training: 某个图片数据集的训练集 (大小32*32*3) 中的属于某些标签的数据（40000 笔）  
* Testing: 此图片数据集中的所有测试数据（10000 笔）  
* Notice: 请勿使用额外数据进行模型训练，亦不可使用预训练模型。可用额外数据辅助验证过程。

## 项目要求

实现以下内容：
1. 方法1-Autoencoder
1. 方法2-K-means
* 假设训练集的标签种类不多
* 假设训练集有 n 群
* 用 K-means 计算训练集中的 n 个 centroid，再用这 n 个 centroid 对训练集分群
* Inlier data 与所分到群的 centroid 的距离应较 outlier 的此距离来得小
1. 方法3-PCA
* 计算训练集的 principal component
* 将测试集投影在这些 component 上
* 再将这些投影重建回原先 space 的向量
* 对重建的图片和原图计算平方差，正常的数值应该较异常的数值为小

## 数据准备

In [ ]:
import numpy as np

train = np.load('data/data58383/train.npy', allow_pickle=True)
test = np.load('data/data58383/test.npy', allow_pickle=True)

# 环境配置/安装
无

# K-means

K-means: 假设训练集的标签种类不多（e.g., < 20），然而因其为未知，可以猜测其为 n，亦即假设训练集有 n 群。先用 K-means 计算训练集中的 n 个 centroid，再用这 n 个 centroid 对训练集分群。应该可以观察到，正常数据与所分到群的 centroid 的距离应较异常数据的此距离来得小。

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import f1_score, pairwise_distances, roc_auc_score
from scipy.cluster.vq import vq, kmeans

if task == 'knn':
    x = train.reshape(len(train), -1)
    y = test.reshape(len(test), -1)
    scores = list()
    for n in range(1, 10):
        kmeans_x = MiniBatchKMeans(n_clusters=n, batch_size=100).fit(x)
        y_cluster = kmeans_x.predict(y)
        print(kmeans_x.cluster_centers_[y_cluster].shape)
        y_dist = np.sum(np.square(kmeans_x.cluster_centers_[y_cluster] - y), axis=1)
        y_pred = y_dist

    #     print('y.shape:', y.shape,'y_pred.shape:', y_pred.shape)
    #     score = f1_score(y, y_pred, average='micro')
    #     # score = roc_auc_score(y_label, y_pred, average='micro')
    #     scores.append(score)
    # print(np.max(scores), np.argmax(scores))
    # print(scores)
    # print('auc score: {}'.format(np.max(scores)))

# PCA

PCA: 首先计算训练集的 principle component，将测试投影在这些 component 上，再将这些投影重建回原先 space 的向量。对重建的图片和原图计算 MSE，正常数据的数值应该较异常数据的数值为小。

In [ ]:
from sklearn.decomposition import PCA

if task == 'pca':

    x = train.reshape(len(train), -1)
    y = test.reshape(len(test), -1)
    pca = PCA(n_components=2).fit(x)

    y_projected = pca.transform(y)
    y_reconstructed = pca.inverse_transform(y_projected)  
    dist = np.sqrt(np.sum(np.square(y_reconstructed - y).reshape(len(y), -1), axis=1))
    
    y_pred = dist
    # score = roc_auc_score(y_label, y_pred, average='micro')
    # score = f1_score(y_label, y_pred, average='micro')
    # print('auc score: {}'.format(score))

# 自编码器模型与损失

* fcn_autoencoder and vae are from https://github.com/L1aoXingyu/pytorch-beginner   
* conv_autoencoder is from https://github.com/jellycsc/PyTorch-CIFAR-10-autoencoder/   

同学们可以尝试各种不同的 VAE 架构

In [ ]:
import paddle
import paddle.nn.functional as F
from paddle.nn import Sequential, Linear, ReLU, Tanh, Conv2D, Conv2DTranspose

class fcn_autoencoder(paddle.nn.Layer):
    def __init__(self):
        super(fcn_autoencoder, self).__init__()
        self.encoder = Sequential(
            Linear(32 * 32 * 3, 128),
            ReLU(),
            Linear(128, 64),
            ReLU(), 
            Linear(64, 12), 
            ReLU(), 
            Linear(12, 3)
            )
        self.decoder = Sequential(
            Linear(3, 12),
            ReLU(),
            Linear(12, 64),
            ReLU(),
            Linear(64, 128),
            ReLU(), 
            Linear(128, 32 * 32 * 3), 
            Tanh()
            )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


class conv_autoencoder(paddle.nn.Layer):
    def __init__(self):
        super(conv_autoencoder, self).__init__()
        self.encoder = Sequential(
            Conv2D(3, 12, 4, stride=2, padding=1),            # [batch, 12, 16, 16]
            ReLU(),
            Conv2D(12, 24, 4, stride=2, padding=1),           # [batch, 24, 8, 8]
            ReLU(),
			Conv2D(24, 48, 4, stride=2, padding=1),           # [batch, 48, 4, 4]
            ReLU(),
            Conv2D(48, 96, 4, stride=2, padding=1),           # [batch, 96, 2, 2]
            ReLU()
        )
        self.decoder = Sequential(
            Conv2DTranspose(96, 48, 4, stride=2, padding=1),  # [batch, 48, 4, 4]
            ReLU(),
			Conv2DTranspose(48, 24, 4, stride=2, padding=1),  # [batch, 24, 8, 8]
            ReLU(),
			Conv2DTranspose(24, 12, 4, stride=2, padding=1),  # [batch, 12, 16, 16]
            ReLU(),
            Conv2DTranspose(12, 3, 4, stride=2, padding=1),   # [batch, 3, 32, 32]
            Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


class VAE(paddle.nn.Layer):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = Linear(32*32*3, 400)
        self.fc21 = Linear(400, 20)
        self.fc22 = Linear(400, 20)
        self.fc3 = Linear(20, 400)
        self.fc4 = Linear(400, 32*32*3)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        eps = paddle.to_tensor(std.shape()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


def loss_vae(recon_x, x, mu, logvar, criterion):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    mse = criterion(recon_x, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = paddle.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return mse + KLD

In [ ]:
import paddle
class TensorDataset(paddle.io.Dataset):
    def __init__(self, tensors, label):
        self.tensors = tensors
        self.label = label

    def __getitem__(self, index):
        return self.tensors[index], paddle.to_tensor([1.])

    def __len__(self):
        return self.tensors.shape[0]

In [8]:
from paddle.io import DataLoader, RandomSampler
from paddle.optimizer import Adam, AdamW
task = 'ae'

if task == 'ae':
    num_epochs = 300 # 1000
    batch_size = 128
    learning_rate = 1e-3

    #{'fcn', 'cnn', 'vae'} 
    model_type = 'cnn' 

    x = train
    if model_type == 'fcn' or model_type == 'vae':
        x = x.reshape(len(x), -1)
        
    data = paddle.to_tensor(x)
    train_dataset = TensorDataset(data, None)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)


    model_classes = {'fcn':fcn_autoencoder(), 'cnn':conv_autoencoder(), 'vae':VAE()}
    model = model_classes[model_type]
    criterion = paddle.nn.loss.MSELoss()
    optimizer = Adam(parameters=model.parameters(), learning_rate=learning_rate)
    
    best_loss = np.inf
    model.train()
    for epoch in range(num_epochs):
        for data in train_dataloader:
            if model_type == 'cnn':
                img = paddle.transpose(data[0], perm=[0, 3, 1, 2])
            else:
                img = data[0]
            # ===================forward=====================
            img = paddle.to_tensor(img, dtype='float32')
            output = model(img)
            if model_type == 'vae':
                loss = loss_vae(output[0], img, output[1], output[2], criterion)
            else:
                loss = criterion(output, img)
            # ===================backward====================
            optimizer.clear_grad()
            loss.backward()
            optimizer.step()
            # ===================save====================
            if loss.numpy()[0] < best_loss:
                best_loss = loss.numpy()[0]
                paddle.save(model.state_dict(), 'best_model_{}.pdparams'.format(model_type))
        # ===================log========================
        print('epoch [{}/{}], loss:{:.4f}'
              .format(epoch + 1, num_epochs, loss.numpy()[0]))

epoch [1/300], loss:0.0608
epoch [2/300], loss:0.0465
epoch [3/300], loss:0.0400


KeyboardInterrupt: 

# 评价

将 testing 的图片输入 model 后，可以得到其重建的图片，并对两者取平方差。可以发现 inlier 的平方差应该与 outlier 的平方差形成差距明显的两群数据。

In [ ]:
if task == 'ae':
    print('model_type:', model_type)
    if model_type == 'fcn' or model_type == 'vae':
        y = test.reshape(len(test), -1)
    else:
        y = test
        
    data = paddle.to_tensor(y, dtype='float32')
    test_dataset = TensorDataset(data, None)
    test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

    layer_state_dict = paddle.load('best_model_{}.pdparams'.format(model_type))
    model.set_state_dict(layer_state_dict)

    model.eval()
    reconstructed = list()
    for i, data in enumerate(test_dataloader): 
        if model_type == 'cnn':
            img = paddle.transpose(data[0], perm=[0, 3, 1, 2])
        else:
            img = data[0]
        output = model(img)
        if model_type == 'cnn':
            output = paddle.transpose(output, perm=[0, 2, 3, 1])
        elif model_type == 'vae':
            output = output[0]
        reconstructed.append(output.detach().numpy())
    reconstructed = np.concatenate(reconstructed, axis=0)
    anomality = np.sqrt(np.sum(np.square(reconstructed - y).reshape(len(y), -1), axis=1))
    y_pred = anomality
    with open('prediction.csv', 'w') as f:
        f.write('id,anomaly\n')
        for i in range(len(y_pred)):
            f.write('{},{}\n'.format(i+1, y_pred[i]))
    # score = roc_auc_score(y_label, y_pred, average='micro')
    # score = f1_score(y_label, y_pred, average='micro')
    # print('auc score: {}'.format(score))

model_type: cnn
